In [15]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


StatementMeta(, 6ce4291b-ee4c-4eb9-8d4f-8870df0a5e49, 17, Finished, Available, Finished)

In [16]:
df_ingredients = spark.read.table("LH_Fastfood.dbo_Ingredients")
df_ingredients = df_ingredients.drop ("IngredientShortDescription")
df_ingredients = df_ingredients.withColumnRenamed("IngredientName", "Ingredient")\
                              .withColumnRenamed("PortionUOMTypeID", "PortionTypeID")
df_ingredients = df_ingredients.dropDuplicates(subset = ["Ingredient"])
df_ingredients = df_ingredients.select("IngredientID", "Ingredient", "PortionTypeID")
df_ingredients.show(10)
df_ingredients.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("LH_Fastfood.dbo_Ingredients")


StatementMeta(, 6ce4291b-ee4c-4eb9-8d4f-8870df0a5e49, 18, Finished, Available, Finished)

+------------+--------------------+-------------+
|IngredientID|          Ingredient|PortionTypeID|
+------------+--------------------+-------------+
|         220|Apple Snapz      ...|            0|
|          18|Pickles          ...|           15|
|         452|Mushrooms        ...|           15|
|         624|white coffee mix ...|           13|
|         516|Hot Pepper Mix   ...|           15|
|          97|Cups, Cold 21 oz ...|           11|
|         147|Straw            ...|           11|
|         491|Thousand Island S...|           15|
|          63|Corned Beef      ...|           15|
|         486|Kung Pao Glaze   ...|           15|
+------------+--------------------+-------------+
only showing top 10 rows



In [17]:
df_transactions = spark.read.table("LH_Fastfood.dbo_OrderTransactions")
df_transactions = df_transactions.dropDuplicates()
df_transactions = df_transactions.withColumnRenamed("MD5KEY_ORDERSALE", "OrderID")
df_transactions = df_transactions.drop("MealLocation")
df_transactions = df_transactions.withColumn("ChangeReceived", df_transactions["ChangeReceived"].cast(FloatType()))\
                                 .withColumn("TaxInclusiveAmount", df_transactions["TaxInclusiveAmount"].cast(FloatType()))\
                                 .withColumn("TaxAmount", df_transactions["TaxAmount"].cast(FloatType()))\
                                 .withColumn("OrderID", lower(df_transactions["OrderID"]))\
                                 .withColumn("Price", col("TaxInclusiveAmount") - col("TaxAmount"))\
                                 .withColumn("date", regexp_replace(df_transactions["date"], "\r", ""))
df_transactions = df_transactions.withColumn("date", date_format(to_date(df_transactions["date"], "yy-MM-dd"), "yyyy-MM-dd"))
df_transactions = df_transactions.select("OrderID", "OrderNumber", "TransactionID", "StoreNumber", "Price", "TaxAmount", "TaxInclusiveAmount", "ChangeReceived", "date")
df_transactions.show(10)
df_transactions.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("LH_Fastfood.dbo_OrderTransactions")


StatementMeta(, 6ce4291b-ee4c-4eb9-8d4f-8870df0a5e49, 19, Finished, Available, Finished)

+--------------------+-----------+-------------+-----------+-----+---------+------------------+--------------+----------+
|             OrderID|OrderNumber|TransactionID|StoreNumber|Price|TaxAmount|TaxInclusiveAmount|ChangeReceived|      date|
+--------------------+-----------+-------------+-----------+-----+---------+------------------+--------------+----------+
|1e10e264ab4f65e75...|      11005|        11005|       4904|  0.0|      0.0|               0.0|          0.15|2015-05-12|
|07076f8916a9f283e...|      11035|        11035|       4904|  0.0|      0.0|               0.0|          0.25|2015-05-13|
|3bf600f0bddd63c58...|       5592|         5592|       4904|  0.0|      0.0|               0.0|         13.25|2015-04-12|
|60b2df151e3affa38...|      13258|        13258|       4904|  0.0|      0.0|               0.0|         13.25|2015-05-28|
|972b87f15ae523f54...|       9133|         9133|       4904|  0.0|      0.0|               0.0|         12.75|2015-05-02|
|56092311c430073c4...|  

In [18]:
df_recipes = spark.read.table("LH_Fastfood.dbo_Recipes")
df_recipes = df_recipes.drop("RecipeName")
df_recipes = df_recipes.withColumnRenamed("RecipeDescription", "Recipe")
df_recipes = df_recipes.dropDuplicates()
df_recipes = df_recipes.select("RecipeID", "Recipe")
df_recipes.show(10)
df_recipes.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("LH_Fastfood.dbo_Recipes")


StatementMeta(, 6ce4291b-ee4c-4eb9-8d4f-8870df0a5e49, 20, Finished, Available, Finished)

+--------+--------------------+
|RecipeID|              Recipe|
+--------+--------------------+
|     245|Steak & Bacon Mel...|
|     496|Bacon Egg & Chees...|
|     754|Pastrami Extr6   ...|
|     857|Bacon Egg & Chees...|
|    1197|Jalapeno Tuna Fla...|
|    1238|Santa Fe Chicken ...|
|    1647|Bkfst BMT CrsMlt ...|
|    2091|BLT BoxLn        ...|
|    2150|B.M.T. Extr6     ...|
|    2909|Sunrise Omlt Wrap...|
+--------+--------------------+
only showing top 10 rows



In [21]:
df_stores = spark.read.table("LH_Fastfood.dbo_Stores")
df_stores = df_stores.drop("StoreType")
df_stores = df_stores.withColumn("StoreAddress", trim(df_stores["StoreAddress"]))\
                     .withColumn("StoreCity", trim(df_stores["StoreCity"]))
df_stores = df_stores.withColumnRenamed("DistributionRegion", "StoreRegion")
df_stores = df_stores.select("StoreNumber","StoreAddress", "StoreRegion", "StoreState", "StoreCity", "StoreZip", "StoreLoyaltyFlag")
df_stores.show()
df_stores.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("LH_Fastfood.dbo_Stores")


StatementMeta(, 6ce4291b-ee4c-4eb9-8d4f-8870df0a5e49, 23, Finished, Available, Finished)

+-----------+--------------+------------+------------+---------+--------+----------------+
|StoreNumber|  StoreAddress| StoreRegion|  StoreState|StoreCity|StoreZip|StoreLoyaltyFlag|
+-----------+--------------+------------+------------+---------+--------+----------------+
|       4904|   Shattuck Sq|Western     |California  | Berkeley|   94704|               Y|
|      12631| Myrtle Avenue|Northeast   |New York    |Ridgewood|   11385|               Y|
|      20974|Whitney Avenue|Northeast   |New York    | Elmhurst|   11373|               Y|
|      46673|  Ninth Street|Western     |California  | Berkeley|   94710|               N|
+-----------+--------------+------------+------------+---------+--------+----------------+



In [22]:
df_SubRecipes = spark.read.table("LH_Fastfood.dbo_SubRecipes")
df_SubRecipes = df_SubRecipes.drop("SubrecipeName")
df_SubRecipes = df_SubRecipes.withColumnRenamed("SubRecipeDescription", "SubRecipe")
df_SubRecipes = df_SubRecipes.select("SubRecipeID", "SubRecipe")
df_SubRecipes.show(10)
df_SubRecipes.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("LH_Fastfood.dbo_SubRecipes")


StatementMeta(, 6ce4291b-ee4c-4eb9-8d4f-8870df0a5e49, 24, Finished, Available, Finished)

+-----------+--------------------+
|SubRecipeID|           SubRecipe|
+-----------+--------------------+
|          1|Std Footlong Sub ...|
|          2|Std 6 Inch Sub Br...|
|          3|Std Footlong Flat...|
|          4|Std 6 Inch Flatbr...|
|          5|Std Salad Lettuce...|
|          6|Std Footlong Vege...|
|          7|Std 6 Inch Vegeta...|
|          8|Std Sliced Cheese...|
|          9|Sliced Cheese    ...|
|         10|Vegetables       ...|
+-----------+--------------------+
only showing top 10 rows



In [23]:
df_sales = spark.read.table("LH_Fastfood.dbo_SalesDetails")
df_sales = df_sales.withColumnRenamed("MD5KEY_MENUITEM", "SalesID")\
                   .withColumnRenamed("MD5KEY_ORDERSALE", "OrderID")\
                   .withColumnRenamed("DepartmentDescription", "Category")\
                   .withColumnRenamed("Description", "MenuItem")
df_sales = df_sales.drop("CategoryDescription")\
                   .drop("PLU")
df_sales = df_sales.withColumn("TaxInclusiveAmount", df_sales["TaxInclusiveAmount"].cast(FloatType()))\
                   .withColumn("TaxAmount", df_sales["TaxAmount"].cast(FloatType()))\
                   .withColumn("AdjustedPrice", df_sales["AdjustedPrice"].cast(FloatType()))\
                   .withColumn("DiscountAmount", df_sales["DiscountAmount"].cast(FloatType()))\
                   .withColumn("Price", df_sales["Price"].cast(FloatType()))\
                   .withColumn("date", date_format(to_date(df_sales["date"], "yy-MM-dd"), "yyyy-MM-dd"))\
                   .withColumn("SalesID", lower(df_sales["SalesID"]))\
                   .withColumn("OrderID", lower(df_sales["OrderID"]))
df_sales = df_sales.dropDuplicates()
df_sales = df_sales = df_sales.select("SalesID", "OrderID", "Price", "TaxAmount", "TaxInclusiveAmount", "DiscountAmount", "AdjustedPrice", "Quantity", "MenuItem", "Category", "MenuItemsID", "date")
df_sales.show(10)
df_sales.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("LH_Fastfood.dbo_SalesDetails")


StatementMeta(, 6ce4291b-ee4c-4eb9-8d4f-8870df0a5e49, 25, Finished, Available, Finished)

+--------------------+--------------------+-----+---------+------------------+--------------+-------------+--------+--------------------+--------------------+-----------+----------+
|             SalesID|             OrderID|Price|TaxAmount|TaxInclusiveAmount|DiscountAmount|AdjustedPrice|Quantity|            MenuItem|            Category|MenuItemsID|      date|
+--------------------+--------------------+-----+---------+------------------+--------------+-------------+--------+--------------------+--------------------+-----------+----------+
|4707a044aceb8e2dc...|0812e661fedfd3cfa...| 0.55|      0.0|               0.0|          0.55|          0.0|       1|Cookie           ...|Cookies          ...|        276|2015-04-12|
|8e17cc83af2a19563...|04cff3d2a8814c516...| 0.65|      0.0|               0.0|          0.65|          0.0|       1|Cookie           ...|Cookies          ...|        276|2015-04-06|
|86b47d993e0fc3c72...|098bc5fb120ccdac5...|  5.0|     0.42|               0.0|          0.

In [24]:
df_menu = spark.read.table("LH_Fastfood.dbo_MenuItems")
df_menu = df_menu.drop("MenuItemName")\
                 .drop("PLU")
df_menu = df_menu.withColumnRenamed("MenuItemDescription", "MenuItem")
df_menu = df_menu.select("MenuItemID", "MenuItem", "RecipeID")
df_menu.show(10)
df_menu.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("LH_Fastfood.dbo_MenuItems")


StatementMeta(, 6ce4291b-ee4c-4eb9-8d4f-8870df0a5e49, 26, Finished, Available, Finished)

+----------+--------------------+--------+
|MenuItemID|            MenuItem|RecipeID|
+----------+--------------------+--------+
|         2|Ham 6 inch       ...|       7|
|         3|Ham FtFbd        ...|       2|
|         5|Ham Salad        ...|       9|
|         6|Ham Mini         ...|      98|
|         7|B.M.T. FtLong    ...|      45|
|         8|B.M.T. 6 inch    ...|      47|
|         9|B.M.T. FtFbd     ...|      46|
|        10|B.M.T. FlatBd    ...|      48|
|        11|B.M.T. Salad     ...|      49|
|        12|Turkey FtLong    ...|      10|
+----------+--------------------+--------+
only showing top 10 rows

